In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

import gc

pd.set_option('display.max_columns', 2000)

Populating the interactive namespace from numpy and matplotlib


In [2]:
DBS_PATH = "../Bases/"

In [3]:
data = pd.read_csv(DBS_PATH + 'games.csv', delimiter=',')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


In [5]:
data.head(3)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3


In [6]:
X = data.drop(columns=["winner"])
y = data["winner"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10029, 15)
(10029, 15)
(10029,)
(10029,)


In [9]:
print(y_train.value_counts())
print(y_test.value_counts())

white    5001
black    4553
draw      475
Name: winner, dtype: int64
white    5000
black    4554
draw      475
Name: winner, dtype: int64


Opening names

In [10]:
opening_names = X[["opening_eco", "opening_name"]].drop_duplicates()

## Feature Engineering

Here we:
- Do the imputations (not needed here, as we have no null values in the columns)
- Create new features based on the dataset we have

In [11]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [150]:
class FeatureEngineering:
    '''
    basic callable methods
    '''
    def __init__(self):
        self.imput_transform = None
        self.create_transform = None
        
    def transform(self, X, y=None):
        X = self.drop_useless_features(X)
        X, y = self.create_features(X, y)
        return X, y
    
    def fit(self, X, y):
        #X = self.imput_nulls(X, y)
        return X, y
    
    def fit_transform(self, X, y):
        X, y = self.fit(X, y)
        X, y = self.transform(X, y)
        return X, y
    
    '''
    auxiliary methods
    '''
    def drop_useless_features(self, X):
        cols_to_drop = [
            'id',
            'white_id',
            'black_id',
            'opening_name'
        ]
        return X.drop(columns=cols_to_drop)
    
    def imput_nulls(self, X, y):
        numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
        categorical_ix = X.select_dtypes(include=['object', 'bool']).columns
        
        t_imput = [
            ('num', SimpleImputer(strategy='median'), numerical_ix), #('num', MinMaxScaler(), numerical_ix)
            ('cat', SimpleImputer(strategy='most_frequent'), categorical_ix)
        ]
        self.imput_transform = ColumnTransformer(transformers=t_imput)
        return self.imput_transform.fit_transform(X, y)
    
    def create_features(self, X, y):
        t_create = [
            ('cat', OneHotEncoder(sparse=False), ['victory_status', 'opening_eco']),
        ]
        self.create_transform = ColumnTransformer(transformers=t_create)
        X_aux = self.create_transform.fit_transform(X, y)
        X_transformed = pd.DataFrame(X_aux, index=X.index, columns=self.create_transform.get_feature_names())
        X_transformed.columns = pd.Series(X_transformed.columns).str.replace("cat__x0", "victory_status").str.replace("cat__x1", "opening_eco")
        X = pd.concat([X, X_transformed], axis=1)
        X = X.drop(columns=['victory_status', 'opening_eco'])
        
        X['created_at'] = pd.to_datetime(X['created_at'], unit='ms')
        X['last_move_at'] = pd.to_datetime(X['last_move_at'], unit='ms')
        X['duration_in_seconds'] = (X['last_move_at'] - X['created_at']) / np.timedelta64(1, 's')
        X['increment_code_minutes'] = X['increment_code'].str.split('+').map(lambda x: x[0]).astype(np.int64)
        X['increment_code_seconds'] = X['increment_code'].str.split('+').map(lambda x: x[1]).astype(np.int64)
        X['rating_difference'] = X['white_rating'] - X['black_rating']
        X.drop(columns=['last_move_at', 'created_at', 'increment_code'], axis=1, inplace=True)        
        print("Shape after creating some features = {}".format(X.shape))

        X = self.create_moves_features(X=X)
        print("Shape after creating moves features = {}".format(X.shape))
        return X, y
    
    def create_moves_features(self, X, period=1, end=None):
        """
        period: frequency of evaluation of the features, in moves (white + black).
        end: last evaluation of the features, in moves (white + black). If None, the longest match size will be taken or 100, minimum.
        """
        # Board notation
        self.colors = ["white", "black"]
        self.pieces = ["R", "N", "B", "Q", "K"]
        self.board_columns = ["a", "b", "c", "d", "e", "f", "g", "h"]
        self.board_rows = ["1", "2", "3", "4", "5", "6", "7", "8"]

        # Get moves
        X["moves"] = X["moves"].apply(lambda x: x.split(" "))
        
        # Get total length
        if (end == None):
            end = 100 #min(np.max(X["moves"].apply(len)), 100)
        print("End move count = {}".format(end))
        
        # We limit the matches sizes to `end`
        X["moves"] = X["moves"].apply(lambda x: x[0:100])
        
        # Apply right pad to matches, using '-'
        X["moves"] = X["moves"].apply(lambda x: self.rpad_matches(x, end, pad='-'))
        
        # Get white and black moves
        X["moves_white"] = X["moves"].apply(lambda x: x[::2])
        X["moves_black"] = X["moves"].apply(lambda x: x[1::2])
        #print("White moves index 0:")
        #print(X["moves_white"].iloc[0])
        #print()
        #print("Black moves index 0:")
        #print(X["moves_black"].iloc[0])
        
        # Prepare all features
        for move_count in range(1, end, period):
            X["moves_white_until_move_count"] = X["moves_white"].apply(lambda x: x[0:move_count])
            X["moves_black_until_move_count"] = X["moves_black"].apply(lambda x: x[0:move_count])
            moves_by_color_until_move_count = {
                "white": X["moves_white_until_move_count"],
                "black": X["moves_black_until_move_count"]
            }

            for color in self.colors:
                X['#P_moves_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: 0 if (x[-1] == '-') else self.qtt_P_moves(x))
                
                X['#center_P_moves_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: 0 if (x[-1] == '-') else self.qtt_center_P_moves(x))
                
                for piece in self.pieces:
                    X['#{}_moves_{}_until_move{}'.format(piece, color, move_count)] = \
                        moves_by_color_until_move_count[color].apply(lambda x: 0 if (x[-1] == '-') else self.qtt_piece_moves(x, piece))
                
                X['castled_kingside_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: False if (x[-1] == '-') else self.castled_kingside(x))
                    
                X['castled_queenside_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: False if (x[-1] == '-') else self.castled_queenside(x))
                    
                X['castled_{}_until_move{}'.format(color, move_count)] = \
                    (X['castled_queenside_{}_until_move{}'.format(color, move_count)] | X['castled_kingside_{}_until_move{}'.format(color, move_count)])
                
                X['#takes_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: 0 if (x[-1] == '-') else self.qtt_takes(x))
                
                X['#checks_{}_until_move{}'.format(color, move_count)] = \
                    moves_by_color_until_move_count[color].apply(lambda x: 0 if (x[-1] == '-') else self.qtt_checks(x))
                
                landing_squares_columns = \
                    moves_by_color_until_move_count[color].apply(self.get_landing_square_columns)
                
                X['#king_sided_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_columns.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_king_sided_moves(x))
                
                X['#queen_sided_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_columns.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_queen_sided_moves(x))
                
                X['#centered_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_columns.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_centered_moves(x))
                
                landing_squares_rows = \
                    moves_by_color_until_move_count[color].apply(self.get_landing_square_rows)
                
                X['#upper_sided_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_rows.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_upper_sided_moves(x, color))
                
                X['#lower_sided_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_rows.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_lower_sided_moves(x, color))
                
                X['#middle_moves_{}_until_move{}'.format(color, move_count)] = \
                    landing_squares_rows.apply(lambda x: 0 if (x[-1] == '-') else self.qtt_middle_moves(x))
            
        X = X.drop(columns=["moves_white", "moves_black", "moves_white_until_move_count", "moves_black_until_move_count"])

        return X
    
    def rpad_matches(self, x, end, pad):
        """
        add right padding character `pad` to all the matches, until the move `end`
        """
        return np.pad(x, (0, end-len(x)), 'constant', constant_values=(pad))
    
    def qtt_P_moves(self, x):
        """
        starts with 'a', ..., 'h'
        """
        return len([move for move in x if move.startswith(tuple(self.board_columns))])
    
    def qtt_center_P_moves(self, x):
        """
        starts with 'd' or 'e'
        """
        return len([move for move in x if move.startswith(tuple(self.board_columns[3:5]))])
    
    def qtt_piece_moves(self, x, piece):
        """
        Obs.: We're not counting castling as King move, even if it's actually a King move.
        """
        return len([move for move in x if move.startswith(piece)]) # starts with piece name
    
    def castled_kingside(self, x):
        """
        contains "O-O"
        """
        return bool(len([move for move in x if (move == 'O-O')]) == 1)
       
    def castled_queenside(self, x):
        """
        contains "O-O-O"
        """
        return bool(len([move for move in x if (move == 'O-O-O')]) == 1)
    
    def qtt_takes(self, x):
        """
        contains 'x'
        """
        return len([move for move in x if 'x' in move])
    
    def qtt_checks(self, x):
        """
        contains '+'
        """
        return len([move for move in x if '+' in move])
    
    def qtt_king_sided_moves(self, x):
        """
        # contains 'e', 'f', 'g' or 'h'
        # + 1 if castled kingside
        """
        return \
            len([col for col in x if col in self.board_columns[4:]]) + \
            int(self.castled_kingside(x))
    
    def qtt_queen_sided_moves(self, x):
        """
        # contains 'a', 'b', 'c' or 'd'
        # + 1 if castled queenside
        """
        return \
            len([col for col in x if col in self.board_columns[:4]]) + \
            int(self.castled_queenside(x))
    
    def qtt_centered_moves(self, x):
        """
        contains 'd' or 'e'
        """
        return len([col for col in x if col in self.board_columns[3:5]])
    
    def qtt_upper_sided_moves(self, x, color):
        """
        the upper side of the board depends on the color:
        # for white player, rows 5 to 8
        # for black player, rows 1 to 4
        """
        rows_range = self.board_rows[4:9] if color == "white" else self.board_rows[0:4]
        return len([row for row in x if row in rows_range])
    
    def qtt_lower_sided_moves(self, x, color):
        """
        the lower side of the board depends on the color:
        # for white player, rows 1 to 4
        # for black player, rows 5 to 8
        """
        rows_range = self.board_rows[0:4] if color == "white" else self.board_rows[4:9]
        return len([row for row in x if row in rows_range])
    
    def qtt_middle_moves(self, x):
        """
        contains '4' or '5'
        """
        return len([row for row in x if row in self.board_rows[3:5]])
    
    ###############################################
    
    def get_landing_square_columns(self, x):
        """
        Get the columns where each move landed.
        Example:
            ["Cf6", "Dxc5+", "e3", "-"] => ["f", "c", "e", "-"]
        """
        return pd.Series(x) \
                 .str.split('#').apply(lambda x: x[0]) \
                 .str.split('+').apply(lambda x: x[0]) \
                 .apply(lambda x: "-" if x[-1] == "-" else x[-2]).to_list()
 
    def get_landing_square_rows(self, x):
        """
        Get the rows where each move landed.
        Example:
            ["Cf6", "Dxc5+", "e3", "-"] => ["6", "5", "3", "-"]
        """
        return pd.Series(x) \
                 .str.split('#').apply(lambda x: x[0]) \
                 .str.split('+').apply(lambda x: x[0]) \
                 .apply(lambda x: x[-1]).to_list()
    
    ###############################################
    
    def show_moves(self, match):
        X = pd.DataFrame()
        X["White"] = match[::2]
        X["Black"] = match[1::2]
        X.index += 1
        return X
        
        
# When modeling:
### in practice, we won't have the fields 'opening_ply', 'last_move_at' and 'turns' completely in a real match. We should have values for these variables in intermediate time of the match
### we should drop 'created_at', once we have created 'games_delay_in_sec'
### we should drop 'black_rating', once we have created 'rating_difference'
### OneHotEncoder should drop_invariant=True
### attention: 'victory_status' == 'outoftime' => may be a draw or a victory => we don't use this variable in modeling!

Test in an aux dataset

In [151]:
X_aux = X_train.sample(2, random_state=42)
y_aux = y_train.sample(2, random_state=42)

In [152]:
X_aux

,id,rated,created_at,last_move_at,turns,victory_status,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
15495,fGpkSBHo,True,1.504466e+12,1.504467e+12,51,outoftime,10+0,caos1968,1428,chrisca408,1544,e4 c5 c4 Nc6 Nf3 d6 d4 cxd4 Nxd4 Nf6 Nxc6 bxc6...,B20,Sicilian Defense: Staunton-Cochrane Variation,3
14401,uwEoLEqt,False,1.503584e+12,1.503586e+12,112,draw,10+10,chess-brahs,1950,aks_oks,1477,Nf3 Nc6 g3 g6 Nc3 Nf6 Bg2 Bg7 d3 d6 e4 e5 Bg5 ...,A04,Zukertort Opening: Black Mustang Defense,2


In [153]:
y_aux

15495    white
14401     draw
Name: winner, dtype: object

In [154]:
fe = FeatureEngineering()

In [155]:
X_new, y_new = fe.fit_transform(X_aux, y_aux)

Shape after creating some features = (2, 14)
End move count = 100
Shape after creating moves features = (2, 3578)


In [158]:
X_new

,rated,turns,white_rating,black_rating,moves,opening_ply,victory_status_draw,victory_status_outoftime,opening_eco_A04,opening_eco_B20,duration_in_seconds,increment_code_minutes,increment_code_seconds,rating_difference,#P_moves_white_until_move1,#center_P_moves_white_until_move1,#R_moves_white_until_move1,#N_moves_white_until_move1,#B_moves_white_until_move1,#Q_moves_white_until_move1,#K_moves_white_until_move1,castled_kingside_white_until_move1,castled_queenside_white_until_move1,castled_white_until_move1,#takes_white_until_move1,#checks_white_until_move1,#king_sided_moves_white_until_move1,#queen_sided_moves_white_until_move1,#centered_moves_white_until_move1,#upper_sided_moves_white_until_move1,#lower_sided_moves_white_until_move1,#middle_moves_white_until_move1,#P_moves_black_until_move1,#center_P_moves_black_until_move1,#R_moves_black_until_move1,#N_moves_black_until_move1,#B_moves_black_until_move1,#Q_moves_black_until_move1,#K_moves_black_until_move1,castled_kingside_black_until_move1,castled_queenside_black_until_move1,castled_black_until_move1,#takes_black_until_move1,#checks_black_until_move1,#king_sided_moves_black_until_move1,#queen_sided_moves_black_until_move1,#centered_moves_black_until_move1,#upper_sided_moves_black_until_move1,#lower_sided_moves_black_until_move1,#middle_moves_black_until_move1,#P_moves_white_until_move2,#center_P_moves_white_until_move2,#R_moves_white_until_move2,#N_moves_white_until_move2,#B_moves_white_until_move2,#Q_moves_white_until_move2,#K_moves_white_until_move2,castled_kingside_white_until_move2,castled_queenside_white_until_move2,castled_white_until_move2,#takes_white_until_move2,#checks_white_until_move2,#king_sided_moves_white_until_move2,#queen_sided_moves_white_until_move2,#centered_moves_white_until_move2,#upper_sided_moves_white_until_move2,#lower_sided_moves_white_until_move2,#middle_moves_white_until_move2,#P_moves_black_until_move2,#center_P_moves_black_until_move2,#R_moves_black_until_move2,#N_moves_black_until_move2,#B_moves_black_until_move2,#Q_moves_black_until_move2,#K_moves_black_until_move2,castled_kingside_black_until_move2,castled_queenside_black_until_move2,castled_black_until_move2,#takes_black_until_move2,#checks_black_until_move2,#king_sided_moves_black_until_move2,#queen_sided_moves_black_until_move2,#centered_moves_black_until_move2,#upper_sided_moves_black_until_move2,#lower_sided_moves_black_until_move2,#middle_moves_black_until_move2,#P_moves_white_until_move3,#center_P_moves_white_until_move3,#R_moves_white_until_move3,#N_moves_white_until_move3,#B_moves_white_until_move3,#Q_moves_white_until_move3,#K_moves_white_until_move3,castled_kingside_white_until_move3,castled_queenside_white_until_move3,castled_white_until_move3,#takes_white_until_move3,#checks_white_until_move3,#king_sided_moves_white_until_move3,#queen_sided_moves_white_until_move3,#centered_moves_white_until_move3,#upper_sided_moves_white_until_move3,#lower_sided_moves_white_until_move3,#middle_moves_white_until_move3,#P_moves_black_until_move3,#center_P_moves_black_until_move3,#R_moves_black_until_move3,#N_moves_black_until_move3,#B_moves_black_until_move3,#Q_moves_black_until_move3,#K_moves_black_until_move3,castled_kingside_black_until_move3,castled_queenside_black_until_move3,castled_black_until_move3,#takes_black_until_move3,#checks_black_until_move3,#king_sided_moves_black_until_move3,#queen_sided_moves_black_until_move3,#centered_moves_black_until_move3,#upper_sided_moves_black_until_move3,#lower_sided_moves_black_until_move3,#middle_moves_black_until_move3,#P_moves_white_until_move4,#center_P_moves_white_until_move4,#R_moves_white_until_move4,#N_moves_white_until_move4,#B_moves_white_until_move4,#Q_moves_white_until_move4,#K_moves_white_until_move4,castled_kingside_white_until_move4,castled_queenside_white_until_move4,castled_white_until_move4,#takes_white_until_move4,#checks_white_until_move4,#king_sided_moves_white_until_move4,#queen_sided_moves_white_until_move4,#centered_moves_white_u

In [148]:
fe.show_moves(X_new["moves"].iloc[0])

,White,Black
1,e4,c5
2,c4,Nc6
3,Nf3,d6
4,d4,cxd4
5,Nxd4,Nf6
6,Nxc6,bxc6
7,Nc3,g6
8,h3,Bg7
9,Qa4,Bd7
10,Qc2,O-O


Now, we apply to the original dataset

Let's do it in chunks:

In [26]:
chunksize = 100
df_shape = X_train.shape[0]

X_train_proc = pd.DataFrame()
y_train_proc = pd.DataFrame()
fe = FeatureEngineering()
for chunk, row in enumerate(range(0,df_shape,chunksize)):
    print("===================")
    print("Chunk {}".format(chunk))
    print("===================")
    print("Positions: [{}:{}]".format(row, row+chunksize-1))
    X_chunk = X_train.iloc[row:row+chunksize]
    y_chunk = y_train.iloc[row:row+chunksize]
    print("Chunk size = {}".format(X_chunk.shape))
    X_chunk_transf, y_chunk_transf = fe.transform(X_chunk, y_chunk)
    X_train_proc = pd.concat([X_train_proc, X_chunk_transf], axis=0)
    y_train_proc = pd.concat([y_train_proc, y_chunk_transf], axis=0)
    print("Final df size = {}".format(X_train_proc.shape))
    print()

Chunk 0
Positions: [0:99]
Chunk size = (100, 15)
Shape after creating some features = (100, 72)
End move count = 100
Shape after creating moves features = (100, 3636)
Final df size = (100, 3636)

Chunk 1
Positions: [100:199]
Chunk size = (100, 15)
Shape after creating some features = (100, 71)
End move count = 100
Shape after creating moves features = (100, 3635)
Final df size = (200, 3664)

Chunk 2
Positions: [200:299]
Chunk size = (100, 15)
Shape after creating some features = (100, 71)
End move count = 100
Shape after creating moves features = (100, 3635)
Final df size = (300, 3688)

Chunk 3
Positions: [300:399]
Chunk size = (100, 15)
Shape after creating some features = (100, 72)
End move count = 100
Shape after creating moves features = (100, 3636)
Final df size = (400, 3706)

Chunk 4
Positions: [400:499]
Chunk size = (100, 15)
Shape after creating some features = (100, 65)
End move count = 100


KeyboardInterrupt: 

In [ ]:
pd.concat([X_train_proc, y_train_proc], axis=0).to_csv(DBS_PATH + "processed_train_games.csv")